<a href="https://colab.research.google.com/github/OnKaa6/Guncel-Proje/blob/main/G%C3%BCncel_Proje_Kodlar%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=1095)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('TSLA',
                   start=start_date,
                   end=end_date,
                   progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.tail())

data = data[["Date", "Close"]]
print(data.head())

import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.figure(figsize=(15, 10))
plt.plot(data["Date"], data["Close"])

from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(data["Close"],
                            model='multiplicative', freq=15)
fig = plt.figure()
fig = result.plot()
fig.set_size_inches(15, 10)

pd.plotting.autocorrelation_plot(data["Close"])

from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(data["Close"], lags=100)

p, d, q = 5, 1, 2
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(data["Close"], order=(p, d, q))
fitted = model.fit(disp=-1)
print(fitted.summary())

predictions = fitted.predict()
print(predictions)

import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

model = sm.tsa.statespace.SARIMAX(data['Close'],
                                  order=(p, d, q),
                                  seasonal_order=(p, d, q, 12))
model = model.fit()
print(model.summary())

predictions = model.predict(len(data), len(data) + 30)
print(predictions)

data["Close"].plot(legend=True, label="Training Data", figsize=(15, 10))
predictions.plot(legend=True, label="Predictions")

import plotly.graph_objects as go
import plotly.express as px

today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=1095)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('TSLA',
                      start=start_date,
                      end=end_date,
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low",
             "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], high=data["High"],
                                        low=data["Low"], close=data["Close"])])
figure.update_layout(title = "Hisse Senedi Fiyat Analizi", xaxis_rangeslider_visible=False)
figure.show()

figure = px.bar(data, x = "Date", y= "Close", title='Hisse Senedi Son 3 Yıllık Kapanış Fiyatları')
figure.show()

figure = px.line(data, x='Date', y='Close',
                 title='Hisse Senedi Fiyat Analizi, Rangeslider Seçeneğiyle')
figure.update_xaxes(rangeslider_visible=True)
figure.show()

figure = px.line(data, x='Date', y='Close',
                 title='Hisse Senedi Fiyat Analizi, Zaman Periyodu Seçenekleriyle')

figure.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
figure.show()

figure = px.scatter(data, x='Date', y='Close', range_x=['2020-01-14', '2023-01-14'],
                 title="Hisse Senedi Fiyat Analizi, Hafta Sonu Boşlukları Gizlenmiş Şekliyle")
figure.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "sun"])
    ]
)
figure.show()